# Retrieving texts and counting words

In this example, we'll retrieve the texts of speeches from a remote server and do a basic word count.

## Scenario

Let's say we want to know how many words Achilles speaks to each of his interlocutors. We can search the DICES database for the relevant speeches using the API. Then, to count the number of words, we'll have to retrieve the text of the speeches themselves. Since the DICES *Speech* objects include CTS URNS, we can request the passages from a remote server. 

## Preliminaries

### The DICES API

First step is to instantiate a connection to the DICES api. 

In [ ]:
from dicesapi import DicesAPI
from dicesapi.jupyter import NotebookPBar
api = DicesAPI(
    progress_class=NotebookPBar,
    logfile='dices.log')

### CTS and CLTK

Text-retrieval and processing tools are moving to the module `text`. Some of this functionality is accessible already from speech objects, but my intention is to remove it. Let's do this the new way:

In [ ]:
from dicesapi.text import CtsAPI
cts = CtsAPI()

### Matplotlib for figures

Let's also import **pyplot**, for drawing a simple bar graph of the results. Note the Jupyter magic `%matplotlib inline` to display the figure right in the notebook. Some people like `%matplotlib notebook` better — it gives you some fancier display options.

In [ ]:
from matplotlib import pyplot
%matplotlib inline

## Running the experiment

Here’s the code for calculating Achilles’ speech lengths by addressee.

### First, download the speeches

Using the API, we can search speeches using a set of key-value pairs. For now, JSON results from the API are paged, so if your search has a lot of results, you may have to wait for several pages to download.

In [ ]:
speeches = api.getSpeeches(spkr_name='Achilles', work_title='Iliad', progress=True)

### Download the text of the speeches from Perseus

This involves retrieving each passage from the CTS server, and extracting the plaintext of its contents.

In [ ]:
pbar = NotebookPBar(max=len(speeches))

# iterate over all speeches
for s in speeches:
    
    # retrieve the passage from the remote library
    s.passage = cts.getPassage(s)
    
    if s.passage is None:
        print(f'Failed to download {speech.urn}')
        
    pbar.update()

### Run CLTK's NLP pipeline

We run a stripped-down version of CLTK's default NLP pipeline, specific to the speech's language.

In [ ]:
pbar = NotebookPBar(max=len(speeches))

# iterate over all speeches
for s in speeches:
    
    # parse with CLTK
    s.passage.runCltkPipeline(index=False)
    
    pbar.update()

### Count words in each speech

`Passage.cltk` gives us acces to the CLTK `Doc` object created by `NLP`.

In [ ]:
count = {}
for s in speeches:
    for addressee in s.addr:
        count[addressee.name] = count.get(addressee.name, 0) + len(s.passage.cltk.words)

### Examine the results

🤔 Let's see whether it worked!

In [ ]:
for name in sorted(count):
    print(name, count[name])

### Make a simple graph with pyplot

Seems good. Let's visualize it with a simple bar chart.

In [ ]:
# data for the graph
names = sorted(count)
y_pos = range(len(names))
bars = [count[name] for name in names]

# create a new figure
fig, ax = pyplot.subplots(figsize=(8, 8))

# draw the bars
ax.barh(y_pos, bars, align='center')

# annotate the graph
ax.set_yticks(y_pos)
ax.set_yticklabels(names)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Number of Words')
ax.set_ylabel('Addressee')
ax.set_title('Length of Achilles\' speeches')